In [102]:
## 1 https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0 
##Auctor: Shashank Kapadia -- 2019  (Testa esse aqui tbm)

## 2 https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html -- doc

##3 https://sigmoidal.ai/como-criar-uma-wordcloud-em-python/  -- WorldClouds ## 
##4 http://tirandolicoesdetudo.com.br/criando-uma-nuvem-de-palavras-wordcloud-com-dados-do-meu-cv/ -- WorldCloud ##

##5 https://www.youtube.com/watch?v=iQ1bfDMCv_c&list=PLf6b7z7NwdGVMXPEvoJu64jxtE58q4CH8&index=3&t=9s - Videos da ALICE ZHAO##
#https://github.com/adashofdata/nlp-in-python-tutorial - 2018


## 6 https://gist.github.com/alopes/5358189 - lista de StopWords em português com correspondências. 


## 7 https://medium.com/@viniljf/utilizando-processamento-de-linguagem-natural-para-criar-um-sumariza%C3%A7%C3%A3o-autom%C3%A1tica-de-textos-775cb428c84e

##8 - https://medium.com/somos-tera/como-modelar-t%C3%B3picos-atrav%C3%A9s-de-latent-dirichlet-allocation-lda-atrav%C3%A9s-da-biblioteca-gensim-1fa17357ad4b -- 2020

In [10]:
#python 3 and python 2 #para puxar as bibliotecas que estejam disponíveis na versões. 
from __future__ import print_function 

In [11]:
# Importing modules 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time 
import sys
import re
%matplotlib inline

In [12]:
#Ler os dados aqui. 
data_df = pd.read_csv('./datasets/docs_gospels_todos.csv', sep=';', encoding='utf-8')
#Imprima as linhas com .head()
data_df.head(10)

,caps,texts
0,Mateus 1,Registro da genealogia de Jesus Cristo filho d...
1,2,A Visita dos Magos 1 Depois que Jesus nasceu e...
2,3,1 Naqueles dias surgiu João Batista pregando n...
3,4,1 Então Jesus foi levado pelo Espírito ao dese...
4,5,As Bem-aventuranças (Lc 6.20-23) 1 Vendo as mu...
5,6,A Ajuda aos Necessitados 1 “Tenham o cuidado d...
6,7,1 “Não julguem para que vocês não sejam julgad...
7,8,1 Quando ele desceu do monte grandes multidões...
8,9,1 Entrando Jesus num barco atravessou o mar e ...
9,10,1 Chamando seus doze discípulos deu-lhes autor...


In [106]:
'''
def combined_text(docs):
    combined_text = ' '.join(docs)
    return combined_text

##Dados combinados dentro da coluna
docs = {key: [combine_text(value)] for (key,value) in docs.items}

pd.set_option('max_colwidth',150)

docs= pd.DataFrame.from_dict(data_combined).transpose()
docs.columns = ['doc']
docs = docs.sort_index()
docs 
'''

"\ndef combined_text(docs):\n    combined_text = ' '.join(docs)\n    return combined_text\n\n##Dados combinados dentro da coluna\ndocs = {key: [combine_text(value)] for (key,value) in docs.items}\n\npd.set_option('max_colwidth',150)\n\ndocs= pd.DataFrame.from_dict(data_combined).transpose()\ndocs.columns = ['doc']\ndocs = docs.sort_index()\ndocs \n"

In [13]:
#Removendo a coluna de capítulos, pois não precisamos das mesmas. 
data_df = data_df.drop('caps', axis=1) #df.drop('column_name', axis=1)
data_df.head()

,texts
0,Registro da genealogia de Jesus Cristo filho d...
1,A Visita dos Magos 1 Depois que Jesus nasceu e...
2,1 Naqueles dias surgiu João Batista pregando n...
3,1 Então Jesus foi levado pelo Espírito ao dese...
4,As Bem-aventuranças (Lc 6.20-23) 1 Vendo as mu...


In [14]:
# Let's take a look at our data again
#next(iter(data_df.keys()))

In [15]:
# Notice that our dictionary is currently in key: comedian, value: list of text format
#next(iter(data_df.values()))

In [16]:
""" 
combined_text=0
def combine_text(doc):
   '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(doc)
    return combined_text

print(combined_text)
"""

" \ncombined_text=0\ndef combine_text(doc):\n   '''Takes a list of text and combines them into one large chunk of text.'''\n    combined_text = ' '.join(doc)\n    return combined_text\n\nprint(combined_text)\n"

In [17]:
# Remove punctuation tutorial 5
import re
import string 

def clean_text_docs_round1(texts):
        texts= texts.lower()
        texts= re.sub('[.*-,/?!:;()\']', ' ',texts)
        texts= re.sub('[-|0:9]', ' ', texts)
        texts= re.sub('[%s]' % re.escape(string.punctuation), ' ',texts)
        texts= re.sub('\w*\d\w*', ' ',texts)
        return texts
    
round1 = lambda x: clean_text_docs_round1(x)
    
# Convert the titles to lowercase
#docs['doc'] = docs['doc'].map(lambda x: x.lower())
#docs['doc'] = docs['doc'].map(lambda x: re.sub('[,\.!?^~´`0:9]', ' ', x))
# Print out the first rows of papers
#docs['doc'].head()


In [18]:
#Atualizar documento com o texto atualizado fase 1
data_clean = pd.DataFrame(data_df.texts.apply(round1))
data_clean.head()

,texts
0,registro da genealogia de jesus cristo filho d...
1,a visita dos magos depois que jesus nasceu e...
2,naqueles dias surgiu joão batista pregando n...
3,então jesus foi levado pelo espírito ao dese...
4,as bem aventuranças lc vendo as mul...


In [19]:
# Apply a second round of cleaning
def clean_text_round2(texts):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    texts = re.sub('[.*‘’“”]', ' ', texts)
    texts = re.sub('\r', ' ', texts)
    texts = re.sub('\n', ' ', texts)
    return texts

round2 = lambda x: clean_text_round2(x)

In [88]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.texts.apply(round2))
data_clean.head(91)

,texts
0,registro da genealogia de jesus cristo filho d...
1,a visita dos magos depois que jesus nasceu e...
2,naqueles dias surgiu joão batista pregando n...
3,então jesus foi levado pelo espírito ao dese...
4,as bem aventuranças lc vendo as mul...
...,...
86,depois de dizer isso jesus olhou para o céu e ...
87,tendo terminado de orar jesus saiu com os seus...
88,jesus é condenado à crucificação então pilat...
89,a ressurreição mt mc lc ...


In [21]:
#Amazenar os dados em um formato pickle para ulilizado posteriormente
#import pickle
#data_clean.to_pickle('dados_limpos.pkl')

In [26]:
from langdetect import detect
from tqdm import tqdm_notebook

In [27]:
data_clean['lang'] = data_clean.texts.progress_map(detect)

In [28]:
data_clean.lang.value_counts()

pt    91
Name: lang, dtype: int64

In [29]:
data_clean = data_clean.loc[data_clean.lang=='pt']

Tokenization

In [41]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Klemer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [90]:
data_clean['sentences'] = data_clean.texts.progress_map(sent_tokenize)

#print(data_clean['sentences'].head(1).tolist()[0][:3]) # Print the first 3 sentences of the 1st article

print(data_clean['sentences'].head(91).tolist()[0][:300])


['registro da genealogia de jesus cristo filho de davi filho de abraão    abraão gerou isaque  isaque gerou jacó jacó gerou judá e seus irmãos   judá gerou perez e zerá cuja mãe foi tamar perez gerou esrom esrom gerou arão   arão gerou aminadabe aminadabe gerou naassom naassom gerou salmom   salmom gerou boaz cuja mãe foi raabe boaz gerou obede cuja mãe foi rute obede gerou jessé   e jessé gerou o rei davi davi gerou salomão cuja mãe tinha sido mulher de urias   salomão gerou roboão roboão gerou abias abias gerou asa   asa gerou josafá josafá gerou jorão jorão gerou uzias   uzias gerou jotão jotão gerou acaz acaz gerou ezequias    ezequias gerou manassés manassés gerou amom amom gerou josias   e josias gerou jeconiasa e seus irmãos no tempo do exílio  na babilônia   depois do exílio na babilônia  jeconias gerou salatiel  salatiel gerou zorobabel   zorobabel gerou abiúde abiúde gerou eliaquim eliaquim gerou azor   azor gerou sadoque sadoque gerou aquim aquim gerou eliúde   eliúde gerou

In [91]:
data_clean['tokens_sentences'] = data_clean['sentences'].progress_map(lambda sentences: [word_tokenize(sentence) for sentence in sentences])
print(data_clean['tokens_sentences'].head(91).tolist())


[[['registro', 'da', 'genealogia', 'de', 'jesus', 'cristo', 'filho', 'de', 'davi', 'filho', 'de', 'abraão', 'abraão', 'gerou', 'isaque', 'isaque', 'gerou', 'jacó', 'jacó', 'gerou', 'judá', 'e', 'seus', 'irmãos', 'judá', 'gerou', 'perez', 'e', 'zerá', 'cuja', 'mãe', 'foi', 'tamar', 'perez', 'gerou', 'esrom', 'esrom', 'gerou', 'arão', 'arão', 'gerou', 'aminadabe', 'aminadabe', 'gerou', 'naassom', 'naassom', 'gerou', 'salmom', 'salmom', 'gerou', 'boaz', 'cuja', 'mãe', 'foi', 'raabe', 'boaz', 'gerou', 'obede', 'cuja', 'mãe', 'foi', 'rute', 'obede', 'gerou', 'jessé', 'e', 'jessé', 'gerou', 'o', 'rei', 'davi', 'davi', 'gerou', 'salomão', 'cuja', 'mãe', 'tinha', 'sido', 'mulher', 'de', 'urias', 'salomão', 'gerou', 'roboão', 'roboão', 'gerou', 'abias', 'abias', 'gerou', 'asa', 'asa', 'gerou', 'josafá', 'josafá', 'gerou', 'jorão', 'jorão', 'gerou', 'uzias', 'uzias', 'gerou', 'jotão', 'jotão', 'gerou', 'acaz', 'acaz', 'gerou', 'ezequias', 'ezequias', 'gerou', 'manassés', 'manassés', 'gerou', 'a

**Lemmatizing with POS tagging**

In [92]:
from nltk import pos_tag

In [93]:
data_clean['POS_tokens'] = data_clean['tokens_sentences'].progress_map(lambda tokens_sentences: [pos_tag(tokens) for tokens in tokens_sentences])
print(data_clean['POS_tokens'].head(91).tolist())


[[[('registro', 'NN'), ('da', 'NN'), ('genealogia', 'NN'), ('de', 'IN'), ('jesus', 'NN'), ('cristo', 'NN'), ('filho', 'NN'), ('de', 'IN'), ('davi', 'FW'), ('filho', 'FW'), ('de', 'FW'), ('abraão', 'FW'), ('abraão', 'FW'), ('gerou', 'FW'), ('isaque', 'NN'), ('isaque', 'NN'), ('gerou', 'NN'), ('jacó', 'NN'), ('jacó', 'NN'), ('gerou', 'NN'), ('judá', 'NN'), ('e', 'NN'), ('seus', 'NN'), ('irmãos', 'NN'), ('judá', 'NN'), ('gerou', 'NN'), ('perez', 'NN'), ('e', 'NN'), ('zerá', 'NN'), ('cuja', 'NN'), ('mãe', 'NN'), ('foi', 'NN'), ('tamar', 'NN'), ('perez', 'NN'), ('gerou', 'NN'), ('esrom', 'IN'), ('esrom', 'JJ'), ('gerou', 'NN'), ('arão', 'NN'), ('arão', 'NN'), ('gerou', 'NN'), ('aminadabe', 'NN'), ('aminadabe', 'NN'), ('gerou', 'NN'), ('naassom', 'JJ'), ('naassom', 'JJ'), ('gerou', 'NN'), ('salmom', 'NN'), ('salmom', 'NN'), ('gerou', 'NN'), ('boaz', 'NN'), ('cuja', 'NN'), ('mãe', 'NN'), ('foi', 'NN'), ('raabe', 'NN'), ('boaz', 'NN'), ('gerou', 'NN'), ('obede', 'VBP'), ('cuja', 'NN'), ('mãe'

In [94]:
# Inspired from https://stackoverflow.com/a/15590384
#https://www.kdnuggets.com/2019/09/overview-topics-extraction-python-latent-dirichlet-allocation.html
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet


def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Klemer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [95]:
# Lemmatizing each word with its POS tag, in each sentence
data_clean['tokens_sentences_lemmatized'] = data_clean['POS_tokens'].progress_map(
    lambda list_tokens_POS: [
        [
            lemmatizer.lemmatize(el[0], get_wordnet_pos(el[1])) 
            if get_wordnet_pos(el[1]) != '' else el[0] for el in tokens_POS
        ] 
        for tokens_POS in list_tokens_POS
    ]
)

In [96]:
print(data_clean['tokens_sentences_lemmatized'].tolist())

[[['registro', 'da', 'genealogia', 'de', 'jesus', 'cristo', 'filho', 'de', 'davi', 'filho', 'de', 'abraão', 'abraão', 'gerou', 'isaque', 'isaque', 'gerou', 'jacó', 'jacó', 'gerou', 'judá', 'e', 'seus', 'irmãos', 'judá', 'gerou', 'perez', 'e', 'zerá', 'cuja', 'mãe', 'foi', 'tamar', 'perez', 'gerou', 'esrom', 'esrom', 'gerou', 'arão', 'arão', 'gerou', 'aminadabe', 'aminadabe', 'gerou', 'naassom', 'naassom', 'gerou', 'salmom', 'salmom', 'gerou', 'boaz', 'cuja', 'mãe', 'foi', 'raabe', 'boaz', 'gerou', 'obede', 'cuja', 'mãe', 'foi', 'rute', 'obede', 'gerou', 'jessé', 'e', 'jessé', 'gerou', 'o', 'rei', 'davi', 'davi', 'gerou', 'salomão', 'cuja', 'mãe', 'tinha', 'sido', 'mulher', 'de', 'urias', 'salomão', 'gerou', 'roboão', 'roboão', 'gerou', 'abias', 'abias', 'gerou', 'asa', 'asa', 'gerou', 'josafá', 'josafá', 'gerou', 'jorão', 'jorão', 'gerou', 'uzias', 'uzias', 'gerou', 'jotão', 'jotão', 'gerou', 'acaz', 'acaz', 'gerou', 'ezequias', 'ezequias', 'gerou', 'manassés', 'manassés', 'gerou', 'am

**Regrouping tokens and removing stop words**

In [98]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
#nltk.download('portuguese')
nltk.download('punkt')
stopwords= nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Klemer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading portuguese: Package 'portuguese' not found
[nltk_data]     in index
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Klemer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from nltk.corpus import stopwords
stopwords_verbs = ['say', 'get', 'go', 'know', 'may', 'need', 'like', 'make', 'see', 'want', 'come', 'take', 'use', 'would', 'can']
stopwords_other = ['de','a', 'o', 'que','e','do', 'da', 'em', 'um', 'para', 'é', 'com', 'não', 'uma', 'os', 
                  'no', 'se','na', 'por','mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem',
                  'à','às','seu', 'sua','ou','ser', 'quando', 'muito', 'há', 'nos', 'já', 'está', 'eu', 
                  'também', 'só','pelo', 'pela','até', 'isso' ,'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 
                  'aos', 'onde','ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'estão', 
                  'você','tinha', 'foram', 'essa','num', 'nem', 'suas', 'meu', 'às', 'minha','têm',
                  'numa', 'pelos', 'elas','havia', 'seja', 'qual', 'será', 'nós', 'tenho', 
                  'lhe', 'deles', 'essas', 'esses','pelas', 'este', 'fosse', 'dele','tu', 'te',
                  'vocês','vos', 'lhes', 'meus', 'minhas', 'teu','tua', 'teus', 'tuas', 'nosso', 
                  'nossa', 'nossos','nossas', 'dela', 'delas', 'esta', 'estes','lhe',
                  'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 
                  'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 
                  'estavam', 'estivera', 'estivéramos', 'esteja','estejamos', 'estejam','estivesse', 'estivéssemos',
                  'estivessem', 'estiver', 'estivermos','estiverem', 'hein', 'há', 'havemos', 'hão', 'houve', 
                  'houvemos', 'houveram', 'houvera','houvéramos', 'haja', 'hajamos', 'hajam','houvesse', 
                  'houvéssemos','houvessem', 'houver', 'houvermos', 'houverem', 'houverei', 
                  'houverá', 'houveremos', 'houverão', 'houveria','houveríamos', 'houveriam' ,'sou' ,'somos',
                  'são' ,'era', 'éramos' ,'eram', 'fui', 'foi', 'fomos', 'foram', 'fora', 'fôramos', 'seja', 
                  'sejamos', 'sejam', 'fosse', 'fôssemos', 'fossem', 'for' ,'formos', 'forem', 'serei', 
                  'será', 'seremos', 'serão','seria','seríamos' ,'seriam' ,'tenho','tem','temos',
                  'têm','tinha' ,'tínhamos' ,'tinham' ,'tive' ,'teve' ,'tivemos','tiveram','tivera','tivéramos',
                  'tenha', 'tenhamos','tenham','tivesse','tivéssemos','tivessem','tiver','tivermos','tiverem',
                  'terei','terá','teremos','terão', 'teria','teríamos','teriam','então','assim', 'pois','digo',
                   'sobre','alguém','alguma','algo','mais longo','muitos','na maioria das vezes','precisando','mais recente',
                   'o mais novo', 'lugar', 'colocar','debaixo','sobre','usar','procurado','seria','não pode','não podia',
                   'não fez','não faz','gerar','gerou','geração','registro', 'registrar','porque', 'respondeu', 
                   'todos', 'mim','ali','lá','onde','lugar','lugares','estava','mc','mt','lc','jo','at','ac','one', 'mr', 
                   'bbc', 'image', 'getty', 'de', 'en', 'caption', 'also', 'copyright', 'something','no','em','na','nuns',
                    
                  ]
my_stopwords = stopwords.words('English') + stopwords_verbs + stopwords_other

In [57]:
from itertools import chain # to flatten list of sentences of tokens into list of tokens

In [58]:
data_clean['tokens'] = data_clean['tokens_sentences_lemmatized'].map(lambda sentences: list(chain.from_iterable(sentences)))
data_clean['tokens'] = data_clean['tokens'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha() 
                                                    and token.lower() not in my_stopwords and len(token)>1])

In [63]:
print(data_clean['tokens'].head().tolist()[0][:300])

['genealogia', 'jesus', 'cristo', 'filho', 'davi', 'filho', 'abraão', 'abraão', 'isaque', 'isaque', 'jacó', 'jacó', 'judá', 'irmãos', 'judá', 'perez', 'zerá', 'cuja', 'mãe', 'tamar', 'perez', 'esrom', 'esrom', 'arão', 'arão', 'aminadabe', 'aminadabe', 'naassom', 'naassom', 'salmom', 'salmom', 'boaz', 'cuja', 'mãe', 'raabe', 'boaz', 'obede', 'cuja', 'mãe', 'rute', 'obede', 'jessé', 'jessé', 'rei', 'davi', 'davi', 'salomão', 'cuja', 'mãe', 'sido', 'mulher', 'urias', 'salomão', 'roboão', 'roboão', 'abias', 'abias', 'asa', 'asa', 'josafá', 'josafá', 'jorão', 'jorão', 'uzias', 'uzias', 'jotão', 'jotão', 'acaz', 'acaz', 'ezequias', 'ezequias', 'manassés', 'manassés', 'amom', 'amom', 'josias', 'josias', 'jeconiasa', 'irmãos', 'tempo', 'exílio', 'babilônia', 'exílio', 'babilônia', 'jeconias', 'salatiel', 'salatiel', 'zorobabel', 'zorobabel', 'abiúde', 'abiúde', 'eliaquim', 'eliaquim', 'azor', 'azor', 'sadoque', 'sadoque', 'aquim', 'aquim', 'eliúde', 'eliúde', 'eleazar', 'eleazar', 'matã', 'mat

In [64]:
"""
# coding: utf8
from __future__ import unicode_literals

 
stop_words = set(""" 
""" 
de 
a 
o 
que 
e 
do 
da 
em 
um 
para 
é 
com 
não 
uma 
os 
no 
se 
na 
por 
mais 
as 
dos 
como 
mas 
foi 
ao 
ele 
das 
tem 
à 
seu 
sua 
ou 
ser 
quando 
muito 
há 
nos 
já 
está 
eu 
também 
só 
pelo 
pela 
até 
isso 
ela 
entre 
era 
depois 
sem 
mesmo 
aos 
ter 
seus 
quem 
nas 
me 
esse 
eles 
estão 
você 
tinha 
foram 
essa 
num 
nem 
suas 
meu 
às 
minha 
têm 
numa 
pelos 
elas 
havia 
seja 
qual 
será 
nós 
tenho 
lhe 
deles 
essas 
esses 
pelas 
este 
fosse 
dele 
tu 
te 
vocês 
vos 
lhes 
meus 
minhas
teu 
tua
teus
tuas
nosso 
nossa
nossos
nossas
dela 
delas 
esta 
estes 
estas 
aquele 
aquela 
aqueles 
aquelas 
isto 
aquilo 
estou
está
estamos
estão
estive
esteve
estivemos
estiveram
estava
estávamos
estavam
estivera
estivéramos
esteja
estejamos
estejam
estivesse
estivéssemos
estivessem
estiver
estivermos
estiverem
hei
há
havemos
hão
houve
houvemos
houveram
houvera
houvéramos
haja
hajamos
hajam
houvesse
houvéssemos
houvessem
houver
houvermos
houverem
houverei
houverá
houveremos
houverão
houveria
houveríamos
houveriam
sou
somos
são
era
éramos
eram
fui
foi
fomos
foram
fora
fôramos
seja
sejamos
sejam
fosse
fôssemos
fossem
for
formos
forem
serei
será
seremos
serão
seria
seríamos
seriam
tenho
tem
temos
tém
tinha
tínhamos
tinham
tive
teve
tivemos
tiveram
tivera
tivéramos
tenha
tenhamos
tenham
tivesse
tivéssemos
tivessem
tiver
tivermos
tiverem
terei
terá
teremos
terão
teria
teríamos
teriam
perguntar
disse
lhes
digo-vos
dizer
havia
alguém
algo
alguma
""" """".split())
"""

 




' \nde \na \no \nque \ne \ndo \nda \nem \num \npara \né \ncom \nnão \numa \nos \nno \nse \nna \npor \nmais \nas \ndos \ncomo \nmas \nfoi \nao \nele \ndas \ntem \nà \nseu \nsua \nou \nser \nquando \nmuito \nhá \nnos \njá \nestá \neu \ntambém \nsó \npelo \npela \naté \nisso \nela \nentre \nera \ndepois \nsem \nmesmo \naos \nter \nseus \nquem \nnas \nme \nesse \neles \nestão \nvocê \ntinha \nforam \nessa \nnum \nnem \nsuas \nmeu \nàs \nminha \ntêm \nnuma \npelos \nelas \nhavia \nseja \nqual \nserá \nnós \ntenho \nlhe \ndeles \nessas \nesses \npelas \neste \nfosse \ndele \ntu \nte \nvocês \nvos \nlhes \nmeus \nminhas\nteu \ntua\nteus\ntuas\nnosso \nnossa\nnossos\nnossas\ndela \ndelas \nesta \nestes \nestas \naquele \naquela \naqueles \naquelas \nisto \naquilo \nestou\nestá\nestamos\nestão\nestive\nesteve\nestivemos\nestiveram\nestava\nestávamos\nestavam\nestivera\nestivéramos\nesteja\nestejamos\nestejam\nestivesse\nestivéssemos\nestivessem\nestiver\nestivermos\nestiverem\nhei\nhá\nhavemos\

In [65]:
""" 
nlp = spacy.load("pt_core_news_sm")

stopwords=STOP_WORDS

texts = data_clean['texts'].tolist()
doc = list(nlp.pipe(texts))

nlp = spacy.load("pt", disable=['parser', 'tagger', 'ner'])
stops = stopwords.words("portuguese")

def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)


data_df['text_after_clean'] = data_clean['texts'].apply(normalize, lowercase=True, remove_stopwords=True)
"""

' \nnlp = spacy.load("pt_core_news_sm")\n\nstopwords=STOP_WORDS\n\ntexts = data_clean[\'texts\'].tolist()\ndoc = list(nlp.pipe(texts))\n\nnlp = spacy.load("pt", disable=[\'parser\', \'tagger\', \'ner\'])\nstops = stopwords.words("portuguese")\n\ndef normalize(comment, lowercase, remove_stopwords):\n    if lowercase:\n        comment = comment.lower()\n    comment = nlp(comment)\n    lemmatized = list()\n    for word in comment:\n        lemma = word.lemma_.strip()\n        if lemma:\n            if not remove_stopwords or (remove_stopwords and lemma not in stops):\n                lemmatized.append(lemma)\n    return " ".join(lemmatized)\n\n\ndata_df[\'text_after_clean\'] = data_clean[\'texts\'].apply(normalize, lowercase=True, remove_stopwords=True)\n'

**WORDCLOUD EXPLORATION**

In [66]:
#Exploratory Analysis
# Import the wordcloud library
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS

#Concatenar as palavras
data_words = " ".join(list(data_clean['texts'].values)) #tutorial 1 

#stopwords=nltk.corpus.stopwords.words('portuguese')

stop_words= set(STOPWORDS) ##Definindo uma lista de STOPWORDS  Tutorial 6
stop_words= my_stopwords

# Criar uma WordCloud # lower max_font_size
wordcloud= WordCloud(stopwords= my_stopwords,background_color="black",  
                     contour_width=3,contour_color="steelblue",
                     max_words=50000,random_state=42, width=1600, height=800,max_font_size=40).generate(data_words)

import matplotlib.pyplot as plt
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
#wordcloud.to_file("WorldCloud_lda_gospels.png") #Salva quando tiver certeza que as imagens estão certas. 


#Gerar a WordCloud
 #for index, doc in enumerate(data_clean.columns):


# Visualizar as WorldCloud
# Mostrar a imagem final 
#import matplotlib.pyplot as plt

#plt.title("Nuvem de palavras dos Evangelhos", fontsize=40, color="red")
#fig, ax=plt.subplots(figsize=(16,6))
#ax.imshow(wordcloud, interpolation="bilinear")
#ax.set_axis_off()
#wordcloud.to_image();
#wordcloud.to_file("WorldCloud_lda_gospels.png") #Salva quando tiver certeza que as imagens estão certas. 

ModuleNotFoundError: No module named 'wordcloud'

**LDA**

**Data preparation**

*Prepare bi-grams and tri-grams*

In [67]:
from gensim.models import Phrases

In [68]:
tokens = data_clean['tokens'].tolist()
bigram_model = Phrases(tokens)
trigram_model = Phrases(bigram_model[tokens], min_count=1)
tokens = list(trigram_model[bigram_model[tokens]])

Prepare objects for LDA gensim implementation

In [69]:
from gensim import corpora

In [70]:
dictionary_LDA = corpora.Dictionary(tokens)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(tok) for tok in tokens]

*Running LDA*

In [71]:
from gensim import models
import numpy as np

In [73]:
np.random.seed(123456)
num_topics = 30
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=4, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

Wall time: 4.72 s


**Quick exploration of LDA results**

Looking at topics

In [205]:
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20):
    print(str(i)+": "+ topic)
    print()

NameError: name 'lda_model' is not defined

**Allocating topics to documents**

In [210]:
print(data_clean.texts.loc[6][:500])

   não julguem para que vocês não sejam julgados    pois da mesma forma que julgarem vocês serão julgados e a medida que usarem também será usada para medir vocês     por que você repara no cisco que está no olho do seu irmão e não se dá conta da viga que está em seu próprio olho    como você pode dizer ao seu irmão   deixe me tirar o cisco do seu olho  quando há uma viga no seu    hipócrita tire primeiro a viga do seu olho e então você verá claramente para tirar o cisco do olho do seu irmão    


In [211]:
#Exibir o lDA MODEL na posiçao desejada
lda_model[corpus[6]] 

NameError: name 'lda_model' is not defined

__Predicting topics on unseen documents__

In [212]:
document = '''não julguem para que vocês não sejam julgados    pois da mesma forma que julgarem vocês serão julgados e a medida que usarem também será usada para medir vocês     por que você repara no cisco que está no olho do seu irmão e não se dá conta da viga que está em seu próprio olho    como você pode dizer ao seu irmão   deixe me tirar o cisco do seu olho  quando há uma viga no seu    hipócrita tire primeiro a viga do seu olho e então você verá claramente para tirar o cisco do olho do seu irmão   '''
tokens = word_tokenize(document)
topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20)
pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])

NameError: name 'lda_model' is not defined

**Advanced exploration of LDA results**

*Allocation of topics in all documents*

In [ ]:
topics = [lda_model[corpus[i]] for i in range(len(data))]

In [ ]:
def topics_document_to_dataframe(topics_document, num_topics):
    res = pd.DataFrame(columns=range(num_topics))
    for topic_weight in topics_document:
        res.loc[0, topic_weight[0]] = topic_weight[1]
    return res

topics_document_to_dataframe([(9, 0.03853655432967504), (15, 0.09130117862212643), (18, 0.8692868808484044)], 20) 

In [216]:
# Like TF-IDF, create a matrix of topic weighting, with documents as rows and topics as columns
document_topic = \
pd.concat([topics_document_to_dataframe(topics_document, num_topics=num_topics) for topics_document in topics]) \
  .reset_index(drop=True).fillna(0)

NameError: name 'topics' is not defined

In [217]:
document_topic.head()

NameError: name 'document_topic' is not defined

In [218]:
# Which document are about topic 14
document_topic.sort_values(14, ascending=False)[14].head(20)

NameError: name 'document_topic' is not defined

In [ ]:
#print(data.articles.loc[91][:1000])

**Looking at the distribution of topics in all documents**

In [ ]:
#%matplotlib inline
#import seaborn as sns; sns.set(rc={'figure.figsize':(10,20)})
#sns.heatmap(document_topic.loc[document_topic.idxmax(axis=1).sort_values().index])

In [ ]:
#sns.set(rc={'figure.figsize':(10,5)})
#document_topic.idxmax(axis=1).value_counts().plot.bar(color='lightblue')

**Visualizing topics**

In [ ]:
# Here a short legend to explain the vis:
# size of bubble: proportional to the proportions of the topics across the N total tokens in the corpus
# red bars: estimated number of times a given term was generated by a given topic
# blue bars: overall frequency of each term in the corpus
# -- Relevance of words is computed with a parameter lambda
# -- Lambda optimal value ~0.6 (https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf)
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary_LDA)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
#LDA com SKlEARN

In [ ]:
#LDA model training and results visualization - tutorial 1 
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA
 
# Helper function
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
# Tweak the two parameters below
number_topics = 30
number_words = 20
# Create and fit the LDA model
lda = LDA(n_components=number_topics, n_jobs=-1)
lda.fit(count_data)
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

In [ ]:
#Analyzing LDA model results -- 8-https://medium.com/somos-tera/como-modelar-topicos-atraves-de-latent-dirichlet-allocation-lda-atraves-da-biblioteca-gensim-1fa17357ad4b
%time
from pyLDAvis import sklearn as sklearn_lda
#iit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if (1 == 1):
    LDAvis_prepared = sklearn_lda.prepare(lda, count_data, count_vectorizer)

with open(LDAvis_data_filepath, 'w') as f:
        pickle.dump(LDAvis_prepared, f)
        
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath) as f:
    LDAvis_prepared = pickle.load(f)
    
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(number_topics) +'.html')